# Collatz conjecture visualizations

Anton Antonov   
RakuForPrediction blog at WordPress   
May 2025

-----

## Introduction

This notebook presents various visualizations related to the [Collatz conjecture](https://en.wikipedia.org/wiki/Collatz_conjecture) using Raku.

The Raku subs for the Collatz sequences are easy to define. The visualizations are done with the packages
["Graph"](https://raku.land/zef:antononcube/Graph),
["JavaScript::D3"](https://raku.land/zef:antononcube/JavaScript::D3), and
["Math::NumberTheory"](https://raku.land/zef:antononcube/Math::NumberTheory).

-----

## Setup

In [ ]:
#%javascript

require.config({
     paths: {
     d3: 'https://d3js.org/d3.v7.min'
}});

require(['d3'], function(d3) {
     console.log(d3);
});

In [ ]:
#%js
js-d3-list-line-plot(rand xx 100, background => '#1F1F1F')

In [ ]:
my $background = 'none';
my $stroke-color = 'Ivory';
my $fill-color = 'none';
my $title-color = 'DarkGray';

In [ ]:
sub darker-shades(Str $hex-color, Int $steps) {
    my @rgb = $hex-color.subst(/ ^ '#'/).comb(2).map({ :16($_) });
    my @shades;
    for 1..$steps -> $step {
        my @darker = @rgb.map({ ($_ * (1 - $step / ($steps + 1))).Int });
        @shades.push: '#' ~ @darker.map({ sprintf '%02X', $_ }).join;
    }
    return @shades;
}

#say darker-shades("#34495E", 5);

In [ ]:
sub blend-colors(Str $color1, Str $color2, Int $steps) {
    my @rgb1 = $color1.subst(/ ^ '#'/).comb(2).map({ :16($_) });
    my @rgb2 = $color2.subst(/ ^ '#'/).comb(2).map({ :16($_) });
    my @blended;

    for ^$steps -> $step {
        my @blend = (@rgb1 Z @rgb2).map({
            ($_[0] + ($step / $steps) * ($_[1] - $_[0])).Int
        });
        @blended.push: '#' ~ @blend.map({ sprintf '%02X', $_ }).join;
    }
    
    return @blended;
}

#say blend-colors("#34495E", "#FFEBCD", 5);

----

## Definition

In [ ]:
sub collatz(UInt $n is copy, Int:D $max-steps = 1000) {
    return [] if $n == 0;
    my @sequence = $n;
    while $n != 1 && @sequence.elems < $max-steps {
        $n = ($n %% 2 ?? $n div 2 !! (3 * $n + 1) / 2).Int;
        @sequence.push: $n;
    }
    return @sequence;
}

In [ ]:
collatz(27)

-----

## Simple visualizations

### Collatz sequence numbers

Here is the simplest -- and very informative -- Collatz sequence plot:

In [ ]:
#% js
js-d3-list-line-plot(collatz(171), :$background)

Let us make a multi-lines plot for a selection of integers:

In [ ]:
my @data = (1..1_000).map({ collatz($_) }).grep({ 30 ≤ $_.elems ≤ 150 && $_.max ≤ 600 }).pick(10).sort(*.head).map({my $i = $_.head; $_.kv.map(-> $x, $y {%(group => $i, :$x, :$y )}).Array }).map(*.Slip).Array;

deduce-type(@data)

**Remark:** Using simple sampling like the code block below would generally produce very non-uniform length and max-member sequences.
Hence we do the filtering above.

```raku
my @data = (^100).pick(9).sort.map(-> $i {collatz($i).kv.map(-> $x, $y {%(group => $i, :$x, :$y )}).Array }).map(*.Slip).Array;
```

In [ ]:
#% js
js-d3-list-line-plot(@data.flat, :$background)

-----

## Sequences lengths distribution

Here is a histogram of the Collarz sequences lengths distributions:

In [ ]:
#% js
my $m = 100_000;
my @cLengths = (^$m).map({ collatz($_).elems });
js-d3-histogram(
    @cLengths, 
    100,
    :$background,
    :600width, 
    :400height, 
    title => "Collatz sequences lengths distribution (up to $m)",
    :$title-color
  )

Here is a corresponding 2D plot (i.e. number vs. sequence length):

In [ ]:
#% js
js-d3-list-plot(
    @cLengths, 
    :$background, 
    :2point-size,
    :800width, 
    :400height, 
    title => 'Collatz sequences lengths',
    x-label => 'integer',
    y-label => 'sequence length',
    :$title-color
  )

-------

## Sunflower embedding

A certain concentric pattern emerges in the spiral embedding plots of the Collatz sequences lengths `mod 8`.

In [ ]:
#% js
my @sunflower = sunflower-embedding(16_000, with => { collatz($_).elems mod 8 mod 3 + 1}):d;

js-d3-list-plot(@sunflower, 
    background => 'none',
    point-size => 4,
    width => 900, height => 440, 
    :!axes, 
    :!legends,
    color-scheme => 'Observable10',
    margins => {:20top, :20bottom, :250left, :250right}
 )

----

## Graphs

Define a sub for graph-edge relationship between consecutive integers in Collatz sequences:

In [ ]:
proto sub collatz-edges(|) {*}

multi sub collatz-edges(Int:D $n) {
    ($n mod 3 == 2) ?? [$n => 2 * $n, $n => (2 * $n - 1) / 3] !! [$n => 2 * $n,]
}

multi sub collatz-edges(@edges where @edges.all ~~ Pair:D) {
    my @leafs = @edges».value.unique;
    @edges.append(@leafs.map({ collatz-edges($_.Int) }).flat)
}

### Small graph

For didactic purposes let use derive the edges of a graph using a certain _small_ number of iterations:

In [ ]:
my @edges = Pair.new(2, 4);

for ^12 { @edges = collatz-edges(@edges) }

deduce-type(@edges)

Make the graph:

In [ ]:
my $g = Graph.new(@edges.map({ $_.value.Str => $_.key.Str })):directed

Plot the graph using suitable embedding:

In [ ]:
#% html
$g.dot(
    engine => 'dot',
    :$background,
    vertex-label-color => 'Gray',
    vertex-shape => 'circle',
    vertex-width => 0.6,
    :24vertex-font-size,
    edge-thickness => 6,
    graph-size => 10
):svg

We clearly follow the Collatz sequence paths in the tree graph.

### Big graph

Let us make a bigger, visually compelling graph:

In [ ]:
#% chat raku
Make progressive blending of two hex colors, like, "#34495E" and "#FFEBCD" for a given total number colors.

In [ ]:
#% html
my @edges = Pair.new(40, 80);
for ^22 { @edges = collatz-edges(@edges) }
my $gBig = Graph.new(@edges.map({ $_.value.Str => $_.key.Str })):!directed;

my @highlight = $gBig.vertex-list.classify({ $_.Int mod 3 }).map(*.value);
my %highlight = <Indigo MediumSlateBlue DarkSlateBlue> Z=> @highlight;
#my %highlight = <#4682B4 #FFD700 #FF6347> Z=> @highlight;
#my %highlight = <#F0FFFF #E6E6FA #B0E0E6> Z=> @highlight;
#my %highlight = <#2C3E50 #34495E #4A6FA5> Z=> @highlight;

$gBig.dot(
    engine => 'neato',
    :%highlight,
    :$background,
    vertex-shape => 'circle',
    vertex-width => 0.45,
    :0vertex-font-size,
    vertex-fill-color => 'Indigo',
    edge-thickness => 8,
    edge-color => 'SlateBlue',
    graph-size => 10
):svg

In [ ]:
my %path-lengths = $gBig.vertex-list.map({ $_ => $gBig.find-path($_, '40').head.elems });

my $d = %path-lengths.values.unique.sort

In [ ]:
#% html
my $root = 64;
my @edges = Pair.new($root, 2 * $root);
for ^22 { @edges = collatz-edges(@edges) }
my $gBig = Graph.new(@edges.map({ $_.value.Str => $_.key.Str })):!directed;
my %path-lengths = $gBig.vertex-list.race(:4degree).map({ $_ => $gBig.find-path($_, $root.Str).head.elems });

%path-lengths.values.unique.elems

In [ ]:
%path-lengths.values.unique.sort

In [ ]:
#%html
my %classes = $gBig.vertex-list.classify({ %path-lengths{$_} });
#my @colors = darker-shades("#FF7F50", %classes.elems).reverse;
#my @colors = blend-colors("#8B0000", "#FFFF00", %classes.elems);
#my @colors = |("#FFA500" xx 16), |blend-colors("#FFA500", "#FFFF00", %classes.elems - 16);
my @colors = |blend-colors("#8B0000", "#FFA500", 16), |blend-colors("#FFA500", "#FFFF00", %classes.elems - 16);
my %highlight = %classes.map({ @colors[$_.key - 1] => $_.value });

$gBig.dot(
    engine => 'neato',
    :%highlight,
    :$background,
    vertex-shape => 'circle',
    vertex-width => 0.4,
    :0vertex-font-size,
    vertex-fill-color => 'Maroon',
    edge-thickness => 8,
    edge-color => 'Purple',
    graph-size => 10
):svg

----

## References

[AAp1] Anton Antonov,
[Graph Raku package](https://github.com/antononcube/Raku-Graph),
(2024-2025),
[GitHub/antononcube](https://github.com/antononcube).

[AAp2] Anton Antonov,
[JavaScript::D3 Raku package](https://github.com/antononcube/Raku-JavaScript-D3),
(2022-2025),
[GitHub/antononcube](https://github.com/antononcube).

[AAp3] Anton Antonov,
[Math::NumberTheory Raku package](https://github.com/antononcube/Raku-Math-NumberTheory),
(2025),
[GitHub/antononcube](https://github.com/antononcube).